Load requirements.py, visualize.py, preprocess.py, feature_extraction.py

In [1]:
!pip install pyphen
from requirements import *
from visualize import *
from preprocess import *
from feature_extraction import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet

In [0]:
def preprocess(stopwords):
  stop_words = set(stopwords.words('english')) 
  default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                    """
  with open('train.json') as json_file:
      data = json.load(json_file)
  return data

# **Dataset**
Load train.json


In [0]:
data = preprocess(stopwords)
essays, age, original_data, age_label = prepare_age(data, 0, [], [], [], [])
essays, age, original_data, age_label = prepare_age(data, 40000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 80000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 120000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 160000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 200000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 240000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 280000, essays, age, original_data, age_label)
essays, age, original_data, age_label = prepare_age(data, 320000, essays, age, original_data, age_label)

# **Dataset summary:**

In [4]:
c13, c14, c15, c16, c1, c2 = age_summary(essays, age, age_label, 0, 0, 0, 0, 0, 0)
print('Number of entries', len(age))

print('13', c13, 'Percent of data', c13/len(age))
print('14', c14, 'Percent of data', c14/len(age))
print('15', c15, 'Percent of data', c15/len(age))
print('16', c16, 'Percent of data', c16/len(age))
print('\n')

print('13-14', c1, 'Percent of data', c1/len(age_label))
print('15-16', c2, 'Percent of data', c2/len(age_label))

Number of entries 79997
13 7211 Percent of data 0.09014088028301061
14 14496 Percent of data 0.18120679525482206
15 21220 Percent of data 0.2652599472480218
16 37070 Percent of data 0.46339237721414556


13-14 21707 Percent of data 0.2713476755378327
15-16 58290 Percent of data 0.7286523244621673


# **Using POS features:**

In [0]:
###------SVM WITH POS_FEATURES-------
import sklearn


classifier = OneVsRestClassifier(SVC(class_weight={0: 2, 1:1}))
batches = [0,40000,80000,120000,160000,200000,240000,280000,320000]
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data, i, [], [], [], [])

  pos_features = POS(essays)
  length = len(essays)
  trainlength = round(0.8*length)
  features = pos_features
  Xtrain = np.asarray(features[:trainlength])
  Ytrain = np.asarray(age_label[:trainlength])

  classifier.fit(Xtrain, Ytrain)
#Xtrain = Xtrain.reshape(-1, 1)

  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

#Xtest = Xtest.reshape(-1, 1)

  pred = classifier.predict(Xtest)
  acc += accuracy_score(Ytest, pred)

  print('Batch', batches.index(i), ':',accuracy_score(Ytest, pred))
print('Overall accuracy:', acc/len(batches))

Batch 0 : 0.7374517374517374
Batch 1 : 0.7269662921348314
Batch 2 : 0.7342419080068143
Batch 3 : 0.7471655328798186
Batch 4 : 0.7403409090909091
Batch 5 : 0.7379619260918253
Batch 6 : 0.7283398546674119
Batch 7 : 0.7323462414578588
Batch 8 : 0.7139664804469273
Overall accuracy: 0.733197875803126


In [0]:
###-----KNN WITH POS_FEATURES---------

clf = neighbors.KNeighborsClassifier(75)

batches = [0,40000,80000,120000,160000,200000,240000,280000,320000]
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data, i, [], [], [], [])
  pos_features = POS(essays)
  length = len(pos_features)
  trainlength = round(0.8*length)
  features = pos_features
  Xtrain = np.asarray(features[:trainlength])
  Ytrain = np.asarray(age_label[:trainlength])

  clf.fit(Xtrain, Ytrain)

  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

#Xtest = Xtest.reshape(-1, 1)

  pred = clf.predict(Xtest)
  acc += accuracy_score(Ytest, pred)
  print('Batch', batches.index(i), ':', accuracy_score(Ytest, pred))
print('Overall accuracy:', acc/len(batches))

Batch 0 : 0.7402095973524545
Batch 1 : 0.7331460674157303
Batch 2 : 0.7421919363997729
Batch 3 : 0.7477324263038548
Batch 4 : 0.740909090909091
Batch 5 : 0.7474804031354984
Batch 6 : 0.7361654555617664
Batch 7 : 0.7329157175398633
Batch 8 : 0.7212290502793296
Overall accuracy: 0.7379977494330402


In [0]:
###-----LINEAR REGRESSION WITH POS_FEATURES---------

clf = LinearRegression()

batches = [0,40000,80000,120000,160000,200000,240000,280000,320000]
mse = 0
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data, i, [], [], [], [])
  pos_features = POS(essays)
  length = len(pos_features)
  trainlength = round(0.8*length)
  features = pos_features
  Xtrain = np.asarray(features[:trainlength])
  Ytrain = np.asarray(age_label[:trainlength])

  clf.fit(Xtrain, Ytrain)

  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

#Xtest = Xtest.reshape(-1, 1)

  pred = clf.predict(Xtest)
  c_pred = np.round(pred)
  mse += mean_squared_error(Ytest, pred)
  acc += accuracy_score(Ytest, c_pred)
  print('Batch', batches.index(i), ':', mean_squared_error(Ytest, pred), 'accuracy:', accuracy_score(Ytest, c_pred))
print('Overall mean squared error:', mse/len(batches))
print('Overall accuracy:', acc/len(batches))

Batch 0 : 0.19065496646321303 accuracy: 0.7402095973524545
Batch 1 : 0.19725587550564172 accuracy: 0.7292134831460674
Batch 2 : 0.18986803600452878 accuracy: 0.7427597955706985
Batch 3 : 0.1866866518154498 accuracy: 0.7477324263038548
Batch 4 : 0.19271158193736018 accuracy: 0.7397727272727272
Batch 5 : 0.18899030530527478 accuracy: 0.7458006718924972
Batch 6 : 0.19623009295556926 accuracy: 0.7350475125768586
Batch 7 : 0.19623277399256908 accuracy: 0.7323462414578588
Batch 8 : 0.20006606474384173 accuracy: 0.7195530726256983
Overall mean squared error: 0.19318848319149426
Overall accuracy: 0.7369372809109686


In [5]:
###---------RANDOM FOREST CLASSIFIER-----------
clf = RandomForestClassifier(n_estimators=4, max_depth=5)

batches = [0,40000,80000,120000,160000,200000,240000,280000,320000]
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data,i, [], [], [], [])
  pos_features = POS(essays)
  length = len(pos_features)
  trainlength = round(0.8*length)
  features = pos_features
  Xtrain = np.asarray(features[:trainlength])
  Ytrain = np.asarray(age_label[:trainlength])

  clf.fit(Xtrain, Ytrain)

  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

#Xtest = Xtest.reshape(-1, 1)

  pred = clf.predict(Xtest)
  acc += accuracy_score(Ytest, pred)
  print('Batch', batches.index(i), ':', accuracy_score(Ytest, pred))
print('Overall accuracy:', acc/len(batches))

Batch 0 : 0.7391064533921676
Batch 1 : 0.7286516853932584
Batch 2 : 0.7433276547416241
Batch 3 : 0.7477324263038548
Batch 4 : 0.7403409090909091
Batch 5 : 0.7446808510638298
Batch 6 : 0.7361654555617664
Batch 7 : 0.7317767653758542
Batch 8 : 0.7201117318435755
Overall accuracy: 0.73687710364076


# **Using n-grams:**

In [8]:
###---------ORIGINAL NGRAMS WITH NAIVE BAYES------------

nb_classifier = MultinomialNB()

batches = []
for i in range(100):
  batches.append(i*1000)
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data, i, [], [], [], [])
  trainlength = round(0.8*len(essays))
  features = original_data

  Xtrain = np.asarray(features[0:trainlength])
  Ytrain = np.asarray(age_label[0:trainlength])
  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

  if i == 0:
    train_data, model = ngram_train(Xtrain, 1 , []) 
  else:
    train_data, model = ngram_train(Xtrain, 1, model)
  test_data = ngram_test(Xtest, model)

  nb_classifier.fit(train_data, Ytrain)
  pred = nb_classifier.predict(test_data)

  acc += accuracy_score(Ytest, pred)
  print('Batch',batches.index(i),':',accuracy_score(Ytest, pred))
print('Overall accuracy:',acc/len(batches))

Batch 0 : 0.7578599007170436
Batch 1 : 0.7575757575757576
Batch 2 : 0.7588300220750552
Batch 3 : 0.7527593818984547
Batch 4 : 0.7615894039735099
Batch 5 : 0.7545706371191135
Batch 6 : 0.7512520868113522
Batch 7 : 0.7426183844011142
Batch 8 : 0.7399777282850779
Batch 9 : 0.7324805339265851
Batch 10 : 0.7259052924791086
Batch 11 : 0.7324022346368715
Batch 12 : 0.7292015633724176
Batch 13 : 0.7329988851727982
Batch 14 : 0.7384529771841959
Batch 15 : 0.7433333333333333
Batch 16 : 0.7357340720221607
Batch 17 : 0.745141588006663
Batch 18 : 0.7386237513873474
Batch 19 : 0.7465258476931629
Batch 20 : 0.7401443642420877
Batch 21 : 0.733222407099279
Batch 22 : 0.7258601553829079
Batch 23 : 0.7288888888888889
Batch 24 : 0.7332962138084632
Batch 25 : 0.7316258351893096
Batch 26 : 0.7423312883435583
Batch 27 : 0.7434613244296049
Batch 28 : 0.7484696716750139
Batch 29 : 0.7427616926503341
Batch 30 : 0.7497206703910615
Batch 31 : 0.7388392857142857
Batch 32 : 0.7371651785714286
Batch 33 : 0.737136465

# **Using text-readability features:**

In [10]:
###------SVM WITH PENTEL FEATURES-------http://ceur-ws.org/Vol-1446/smlir_submission2.pdf
classifier = OneVsRestClassifier(SVC())
batches = [0,40000,80000,120000,160000,200000,240000,280000]
acc = 0
for i in batches:
  essays, age, original_data, age_label = prepare_age(data, i, [], [], [], [])

  features, age_label = extract_features(essays, age_label)
  length = len(features)
  trainlength = round(0.8*length)
  Xtrain = np.asarray(features[:trainlength])
  Ytrain = np.asarray(age_label[:trainlength])

  classifier.fit(Xtrain, Ytrain)
#Xtrain = Xtrain.reshape(-1, 1)

  Xtest = np.asarray(features[trainlength:])
  Ytest = np.asarray(age_label[trainlength:])

#Xtest = Xtest.reshape(-1, 1)

  pred = classifier.predict(Xtest)

  acc += accuracy_score(Ytest, pred)
  print('Batch', batches.index(i), ':',accuracy_score(Ytest, pred))
print('Overall accuracy:', acc/len(batches))

Batch 0 : 0.7386489479512736
Batch 1 : 0.7367231638418079
Batch 2 : 0.7482876712328768
Batch 3 : 0.7472965281730222
Batch 4 : 0.7414383561643836
Batch 5 : 0.7467642093415869
Batch 6 : 0.7365168539325843
Batch 7 : 0.7332570120206068
Overall accuracy: 0.7411165928322677


# **Analysis on HP dataset:**
Load training_set_rel3.tsv (in asap-aes folder) and age_pentel_model.sav

In [2]:
#Parth's code
%matplotlib inline

dataset_path = "training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1", index_col="essay_id")
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]

essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)

data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
print("All Data:", len(data))
data.head()

Set 1 : Essays =  1783 	 Attributes =  5
Set 2 : Essays =  1800 	 Attributes =  8
Set 3 : Essays =  1726 	 Attributes =  5
Set 4 : Essays =  1770 	 Attributes =  5
Set 5 : Essays =  1805 	 Attributes =  5
Set 6 : Essays =  1800 	 Attributes =  5
Set 7 : Essays =  1569 	 Attributes =  13
Set 8 : Essays =  723 	 Attributes =  17
All Data: 12976


/content/feature_extraction.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)


,essay_set,essay,domain1_score
essay_id,,,
1,1,"Dear local newspaper, I think effects computer...",8
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
5,1,"Dear @LOCATION1, I know having computers has a...",8


In [0]:
#Parth's code
#Dataset keys
essay_id_key = "essay_id"
essay_set_key = "essay_set"
essay_key = "essay"
domain1_score_key = "domain1_score"

In [4]:
all_essays = []
all_scores = []
norm_score = []
y = []

classifier = pickle.load(open('age_pentel_model.sav', 'rb'))
c1 = 0
c2 = 0

for idx in range(len(data)):
  all_essays.append(data.iloc[idx][essay_key])
  all_scores.append(data.iloc[idx][domain1_score_key])
  set_id = data.iloc[idx][essay_set_key]
  if set_id == 7 or set_id == 1 or set_id == 5: #grades 7-8
    y.append(1)
    c1 += 1
  elif set_id == 2 or set_id == 3 or set_id == 4 or set_id == 6 or set_id == 8:
    y.append(2)
    c2 += 1
  sc = data.iloc[idx][domain1_score_key]
  if set_id == 1:
    norm_score.append(sc/12)
  elif set_id == 2:
    norm_score.append(sc/6)
  elif set_id == 3 or set_id == 4:
    norm_score.append(sc/3)
  elif set_id == 5 or set_id == 6:
    norm_score.append(sc/4)
  elif set_id == 7:
    norm_score.append(sc/30)
  else:
    norm_score.append(sc/60)





print('Grade 7-8:', c1)
print('Grade 9-10:', c2)

e = prepare_test(all_essays)
e = np.asarray(e)

#e_feats = POS(e)
e_feats = pentel_features(e)
y = np.asarray(y)

pred = classifier.predict(e_feats)
y_pred = np.round(pred)
print('Accuracy score on HP dataset:',accuracy_score(y, y_pred))

score1 = 0
score2 = 0
count1 = 0
count2 = 0
count3 = 0
score3 = 0

f1 = np.zeros(len(e_feats[0]))
f2 = np.zeros(len(e_feats[0]))
f3 = np.copy(f2)

for i in range(len(e_feats)):
  if y[i] == 1: 
    score1 += norm_score[i]
    count1 += 1
    f1 += e_feats[i]
  else:
    score2 += norm_score[i]
    count2 += 1
    f2 += e_feats[i]
  if norm_score[i] > 0.8:
    f3 += e_feats[i]
    score3 += norm_score[i]
    count3 += 1

print('Observing bias:\n')
print('7-8 graders average score:', score1/count1)
print('9-10 graders average score:', score2/count2)

f1 = f1/count1
f2 = f2/count2

,
print('\n')
print('Feature:                                     ', 'Avg value for 7-8 graders    ', 'Avg value for 9-10 graders')
print('Avg no of characters in a word               ', f1[0],'                       ', f2[0])
print('Avg no of words in an entry                  ', f1[1],'                       ', f2[1])
print('Avg ratio of complex words to all words      ', f1[2],'                       ', f2[2])
print('Avg number of syllables per word             ', f1[3],'                       ', f2[3])
print('Avg ratio of 1-syllable words to all words   ', f1[4],'                       ', f2[4])
print('Avg ratio of 2-syllable words to all words   ', f1[5],'                       ', f2[5])
print('Avg ratio of 3-syllable words to all words   ', f1[6],'                       ', f2[6])
print('Avg ratio of 4-syllable words to all words   ', f1[7],'                       ', f2[7])
print('Avg ratio of 5-syllable words to all words   ', f1[8],'                       ', f2[8])
print('Avg ratio of 6-syllable words to all words   ', f1[9],'                       ', f2[9])
print('Avg ratio of 7-syllable words to all words   ', f1[10],'                      ', f2[10])
print('Avg ratio of 8-syllable words to all words   ', f1[11],'                                        ', f2[11])

print('\n Average feature for a score higher than 0.8:\n')
f3 = f3/count3
print('Avg no of characters in a word             ', f3[0])
print('Avg no of words in an entry                ', f3[1])
print('Avg ratio of complex words to all words     ', f3[2])
print('Avg number of syllables per word            ', f3[3])
print('Avg ratio of 1-syllable words to all words ', f3[4])
print('Avg ratio of 2-syllable words to all words', f3[5])
print('Avg ratio of 3-syllable words to all words', f3[6])
print('Avg ratio of 4-syllable words to all words', f3[7])
print('Avg ratio of 5-syllable words to all words', f3[8])
print('Avg ratio of 6-syllable words to all words ', f3[9])
print('Avg ratio of 7-syllable words to all words', f3[10])
print('Avg ratio of 8-syllable words to all words', f3[11])

Grade 7-8: 5157
Grade 9-10: 7819
Accuracy score on HP dataset: 0.6025739827373613
Observing bias:

7-8 graders average score: 0.6193975825738465
9-10 graders average score: 0.5885982862258675


Feature:                                      Avg value for 7-8 graders     Avg value for 9-10 graders
Avg no of characters in a word                4.213243290946323                         4.346382762791239
Avg no of words in an entry                   220.93368237347295                         224.85829389947563
Avg ratio of complex words to all words       0.01207175405801296                         0.016911816334866284
Avg number of syllables per word              0.3351132753186762                         0.3371852009780049
Avg ratio of 1-syllable words to all words    0.1801288431016428                         0.18407552350903708
Avg ratio of 2-syllable words to all words    0.057745837386756635                         0.05004014451095775
Avg ratio of 3-syllable words to all words    0.00